In [1]:
import numpy as np
import pandas as pd
from clean import manifest_clinical_merge
from collections import Counter

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
manifest_df = pd.read_csv('../Manifest_Data/GCD_TARGET_Data_Manifest_AML_NBL_WT_RT.csv')

wt_disc_df = pd.read_excel('../Clinical_Data/TARGET_WT_ClinicalData_Discovery_20160714_public.xlsx')
aml_disc_df = pd.read_excel('../Clinical_Data/TARGET_AML_ClinicalData_20160714.xlsx')
nbl_disc_df = pd.read_excel('../Clinical_Data/TARGET_NBL_ClinicalData_20151124.xlsx')

WT_df = manifest_clinical_merge(manifest_df, wt_disc_df, 'TARGET-WT')
AML_df = manifest_clinical_merge(manifest_df, aml_disc_df, 'TARGET-AML')
NBL_df = manifest_clinical_merge(manifest_df, nbl_disc_df, 'TARGET-NBL')

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/clean.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  target_df['TARGET USI'] = target_df.loc[:, 'entity_submitter_id'].apply(lambda x: x[:16])


In [4]:
from clean import assay_transpose

In [5]:
assay_df = pd.read_csv('../Expn_Data/TARGET_NBL_AML_RT_WT_TMMCPM_log2_Norm_Counts.csv.zip')
assay_t_df = assay_transpose(assay_df)

In [6]:
from clean import assay_clinical_merge

In [7]:
AML_genes = assay_clinical_merge(assay_t_df, AML_df)
WT_genes = assay_clinical_merge(assay_t_df, WT_df)
NBL_genes = assay_clinical_merge(assay_t_df, NBL_df)

In [8]:
from model_comp import data_prep_columns, model_prep, model_comp, model_prep_loc, xgboost_tuner

In [9]:
df, __ = data_prep_columns(AML_genes, 'neither')

In [10]:
df.describe()

,Age at Diagnosis in Days,Event Free Survival Time in Days,Overall Survival Time in Days,Year of Diagnosis,Year of Last Follow Up,WBC at Diagnosis,FLT3/ITD allelic ratio,MRD % at end of course 2,file_size,Low Risk
count,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,11.000000,108.000000,137.000000,137.000000
mean,3427.153285,967.321168,1616.408759,2006.131387,2010.532847,70.596350,0.471818,0.817870,255089.934307,0.437956
std,2190.723406,973.461901,1062.334254,2.925088,3.636116,82.214077,0.245146,5.653352,5125.026215,0.497956
min,137.000000,77.000000,112.000000,1996.000000,1997.000000,1.300000,0.150000,0.000000,237920.000000,0.000000
25%,1248.000000,284.000000,536.000000,2005.000000,2009.000000,18.200000,0.220000,0.000000,252131.000000,0.000000
50%,3539.000000,461.000000,1712.000000,2007.000000,2011.000000,44.200000,0.540000,0.000000,255170.000000,0.000000
75%,5346.000000,1798.000000,2509.000000,2008.000000,2013.000000,94.800000,0.635000,0.000000,258251.000000,1.000000
max,8231.000000,3630.000000,4022.000000,2010.000000,2015.000000,519.000000,0.830000,57.000000,270306.000000,1.000000


# DATA CLEANED

In [11]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from xgboost import XGBClassifier

In [34]:
model_columns = df.iloc[:, 84:-4].columns
X = df[model_columns].astype(float)
y = df.iloc[:, -1]

In [14]:
print(len(X))
len(X) == len(y)

137


True

In [43]:
df.head()

,TARGET USI,Diagnostic ID,Gender,Race,Ethnicity,Age at Diagnosis in Days,First Event,Event Free Survival Time in Days,Vital Status,Overall Survival Time in Days,Year of Diagnosis,Year of Last Follow Up,Protocol,WBC at Diagnosis,Bone marrow leukemic blast percentage (%),Peripheral blasts (%),CNS disease,Chloroma,FAB Category,t(6;9),t(8;21),t(3;5)(q25;q34),t(6;11)(q27;q23),t(9;11)(p22;q23),t(10;11)(p11.2;q23),t(11:19)(q23:p13.1),inv(16),del5q,del7q,del9q,monosomy 5,monosomy 7,trisomy 8,trisomy 21,MLL,Minus Y,Minus X,Cytogenetic Code Other,Cytogenetic Complexity,Primary Cytogenetic Code,ISCN,FLT3/ITD positive?,FLT3/ITD allelic ratio,FLT3 PM,NPM mutation,CEBPA mutation,WT1 mutation,c-Kit Mutation Exon 8,c-Kit Mutation Exon 17,MRD at end of course 1,...,ENSG00000280913.1,ENSG00000280916.1,ENSG00000280927.1,ENSG00000280936.1,ENSG00000280953.1,ENSG00000280987.1,ENSG00000281005.1,ENSG00000281021.1,ENSG00000281026.1,ENSG00000281100.1,ENSG00000281103.1,ENSG00000281106.1,ENSG00000281128.1,ENSG00000281131.1,ENSG00000281183.1,ENSG00000281189.1,ENSG00000281195.1,ENSG00000281205.1,ENSG00000281207.1,ENSG00000281332.1,ENSG00000281344.1,ENSG00000281357.1,ENSG00000281358.1,ENSG00000281376.1,ENSG00000281383.1,ENSG00000281392.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281406.1,ENSG00000281433.1,ENSG00000281445.1,ENSG00000281468.1,ENSG00000281490.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281538.1,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1,__no_feature,__ambiguous,__alignment_not_unique,Low Risk
3,TARGET-20-PADZCG,09A,Female,Unknown,Not Hispanic or Latino,5325,Relapse,314,Dead,536,1996,1998,CCG-2961,29.3,58,5,No,No,M4,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,0,Normal,"46,XX[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-2.42284,0.314777,1.74517,-6.92714,-6.12136,1.40683,0.995511,-0.223406,2.2914,-0.613341,3.9011,1.61224,-4.6821,-6.92714,0.8846,-2.4712,0.732776,0.678398,0.391151,1.82242,1.4786,1.37983,0.894175,-1.11056,-0.330849,0.0915962,1.95643,-0.547052,-6.92714,-1.45498,1.12314,-2.93057,2.9388,1.76878,0.580973,-0.832714,-1.78095,0.132623,-0.682824,5.46561,-0.330849,-1.84323,-0.801464,-0.364707,2.1646,-0.152689,17.1243,14.6061,17.8527,0
5,TARGET-20-PAEAKL,09A,Female,Black or African American,Not Hispanic or Latino,5218,Relapse,134,Dead,163,1997,1997,CCG-2961,96.1,80,81,Yes,No,M5,No,No,No,No,Yes,No,Yes,No,No,No,No,No,No,No,No,Yes,No,No,,1,MLL,"46,XX,t(11:19)(q23:p13.1)[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-5.80459,-6.92714,1.03526,-6.92714,-6.92714,0.113774,1.32433,-4.41384,0.664587,-6.92714,1.21435,-6.92714,-6.92714,-5.80459,-0.0939682,-2.53384,-1.48807,-4.14335,-1.05214,0.929309,0.340124,0.95113,1.78753,-0.202203,6.32096,-0.544956,1.04881,0.496696,-6.92714,-2.79931,-0.914236,-1.94482,1.12118,3.16766,-2.61702,-1.89207,-3.1249,1.05554,-1.94482,3.83862,-2.17756,-0.717812,-0.88817,1.75482,0.456577,0.965496,16.7942,15.3635,19.6702,0
9,TARGET-20-PAEERJ,09A,Female,White,Not Hispanic or Latino,5082,Relapse,259,Dead,373,1997,1998,CCG-2961,71.9,60,41,No,No,M4,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,1,Other,"46,XX,t(3:5)(q23:q33)[19]/46,XX[1]",Yes,0.55,No,No,No,Yes,Not done,Not done,Unknown,...,-3.82289,-3.00939,2.17987,-6.92714,-6.92714,1.94937,-1.00327,0.722174,3.02575,-3.82289,4.43513,-3.49809,-6.92714,-6.92714,1.32028,-4.50961,1.67087,-2.35434,-0.680935,1.47645,1.32543,0.512625,0.884345,-0.488087,-1.00327,-1.25643,2.91237,-0.209853,-6.92714,-1.35175,-0.0681887,-2.81572,-0.0817453,0.557041,0.30953,-2.56667,-1.85807,-0.470147,1.27314,6.07709,0.768241,-0.600677,-1.56366,-0.506252,2.3282,-2.90931,16.916,14.5652,17.571,0
10,TARGET-20-PAEFGR,09A,Female,White,Not Hispanic or Latino,549,Relapse,497,Alive,4022,1997,2008,CCG-2961,51.2,75,41,No,No,M2,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,,1,inv(16),"46,XX,inv(16)(p13q22)[21]/46,XX[1]",No,NaN,No,No,No

In [35]:
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=700,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=8,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict_proba(X_test)
xgb_ll = log_loss(y_test, xgb_pred)
xgb_ll

0.16476877510015453

Log Loss: 0.16476877510015453

In [51]:
preds = xgb_pred[:, 1]

In [56]:
def permutation_test(pred, y, n):
    actual_loss = log_loss(y, pred)
    losses = []
    pred = pred.copy()
    for _ in range(n):
        np.random.shuffle(pred)
        losses.append(log_loss(y, pred))
    return losses

In [69]:
run1 = permutation_test(preds, y_test, 50000)

In [70]:
[ x for x in run1 if x < 0.164768]

[]

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict_proba(X_test)
xgb_ll = log_loss(y_test, xgb_pred)
preds = xgb_pred[:, 1]
xgb_ll

0.4814516596762197

In [71]:
run2 = permutation_test(preds, y_test, 50000)

In [72]:
[ x for x in run2 if x < 0.4814516]

[0.4767559113247054, 0.3691962517265763]

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict_proba(X_test)
xgb_ll = log_loss(y_test, xgb_pred)
preds = xgb_pred[:, 1]
xgb_ll

0.3707692888432315

In [74]:
run3 = permutation_test(preds, y_test, 50000)

In [75]:
[ x for x in run3 if x < 0.5038286]

[]

In [77]:
xgb_features = X.columns[np.array(xgb.feature_importances_) > 0]
len(xgb_features)

291

In [79]:
# xgb_features